# Basic homomorphic encryption
Let's start with some basic homomorphic encryption using exponentiation of a generator and modulo

The first step is to get a polynomial computed in the unencrypted domain, and in the encrypted domain

$$ p(x)= c0 + c1*x + c2*x^2 +c3*x^3 $$

The other way to represent this polynomial is by looking using co-factors, which uses the roots (solution) of the polynomial.

$$ p(x) = (x - r1) * (x - r2) * (x - r3) = 0 $$


In [72]:
from IPython.display import Markdown as md
import numpy as np
# Values for the user to input, if he wants to
generator = 19
field_prime = 23

# x^3 - 18 x^2 + 80 x - 96 = 0
# Co-factors (roots) for this polynomial are [2,4,12]

# Let's compute the coefficents from known roots and see what we get. 
# We'll use numpy's integrated function to do this
polynomial_roots = [10,3,5]
calculated_coefficients = np.polynomial.polynomial.polyfromroots(polynomial_roots)
print(calculated_coefficients)
#p_polynomial_coefficents = [96,80,-18,1]
p_polynomial_coefficents = calculated_coefficients.astype(int)
target_polynomial_coefficents = []

# change_variables = input("Do you want to change variables?")
# if change_variables == "yes":
#     generator = input("generator")

equation = "$$ p(x)= "

for index, coeff in enumerate(p_polynomial_coefficents):

    if index == 0:
        equation+= str(coeff)
    else:
        if coeff >= 0:
            equation+= (" + ")
        else:
            equation+= (" - ")
        equation+= str(abs(coeff))
        equation+= (" * x^{}".format(index))
               
equation+= (" $$")
md(equation)
#md("$$ p(x)= {} + {}*x + c2*x^2 $$".format(p_polynomial_coefficents[0],p_polynomial_coefficents[1]))

[-150.   95.  -18.    1.]


$$ p(x)= -150 + 95 * x^1 - 18 * x^2 + 1 * x^3 $$

In [73]:
# define the encryption function
# all operations defined on Galois field G(p), with p being a prime number
# encryption is done by exponentiation of a generator (a prime number too) to the power of the unencrypted value
# Now fortunately, the pow() function in Python has an embedded functionality to apply a modulo
#  and it even works on large numbers, by using reduction
def encrypt(unencrypted, gen=generator,modulo=field_prime):
    encrypted = pow(gen,unencrypted,modulo)
    #print("exponentiated to : ", encrypted)
    return encrypted
    
print("encrypt 5, with generator 7 and p 19: ",encrypt(5))
print("encrypt 6, with generator 7 and p 19: ",encrypt(6))
print("encrypt 334499, with generator 7 and p 19: ",encrypt(334499))
print('let\'s what we get with negative number -5,',encrypt(-5))


encrypt 5, with generator 7 and p 19:  11
encrypt 6, with generator 7 and p 19:  2
encrypt 334499, with generator 7 and p 19:  22
let's what we get with negative number -5, 21


## Now we see how we can do our first encryption of polynomial
The way to do it is first to compute the polynomial in the unencrypted domain for a given r(a scalar that is used do compute the value of the polynomial, e.g. p(r)), then encrypt this result using the method described above.
Then the prover computes it in the encrypted domain, returns it to the verifier.
Finally, the Verifier verifies the 2 values match.

In [ ]:
# We have defined the polynomial earlier with its coefficients in calculated_coefficients
# let's compute the polynomial p(r) value with r = 11 (not a root)
r=21
evaluated_poly = np.polynomial.polynomial.polyval(r, p_polynomial_coefficents)
print('the polynomial evaluated at r: ',evaluated_poly)
# Now let's encrypt this value and see what we get
encrypted_result = encrypt(int(evaluated_poly))
print('encrypted result: ',encrypted_result)

# The second part is to verify that we can get the same result when doing the computation in the encryted domain
# How we do this is by having the verifier (he chooses a secret value where to evaluate the polynomial) give the encrypted value
# of each x^i for the value r. In our case the prover needs the encrypted values for 11^3, 11^2, 11^1 and 11^0
# For now we just want to see it in action, we are not yet implementing the protocol for Zero Knowledge proof
encrypted_powers = []
for index, order in enumerate(p_polynomial_coefficents):
    encrypted_powers.append(encrypt(pow(r,index)))

# Now let's compute E (p(11)) = encrypted_powers[0]^coeff[0] * encrypted_powers[1]^coeff[1] * encrypted_powers[2]^coeff[2] * encrypted_powers[3]^coeff[3] 
# We apply modulo a few times along the way to reduce the chance of overflow
result = 1
for index,coeff in enumerate(p_polynomial_coefficents):
    result = (result * pow(encrypted_powers[index],int(coeff),field_prime)) % field_prime
print('the result is: ....',result)

# I think from here on out it would be good to have a verifier functin and a prover function and clearly delineates the tasks they each
# must perform. As the functionality will evolve as we make the protocol more complex, the functions will be named 
# using the chapter number defining the protocol

# We will need random values from here on out, let's use the secrets module for it
import secrets

def verifier_334():
# The verifier's tasks are as follows:
# (a) sample a random value s, i.e., secret
# (b) calculate encryptions of s for all powers i in 0, 1, ..., d, i.e.: E(si) = gsi
# (c) evaluate unencrypted target polynomial with s: t(s)
# (d) encrypted powers of s are provided to the prover: E(s0), E(s1), ..., E(sd)

    s = secrets.randbelow(field_prime) # (a)

    encrypted_powers = [] # (b) & (d)
    for index, order in enumerate(p_polynomial_coefficents):
        encrypted_powers.append(encrypt(pow(s,index)))

     # let's compute the polynomial p(r) value with r = 11 (not a root)
    evaluated_poly2 = np.polynomial.polynomial.polyval(11, calculated_coefficients)
    print('the polynomial evaluated at 11: ',evaluated_poly2)
    # Now let's encrypt this value and see what we get
    encrypted_result2 = encrypt(int(evaluated_poly2))
    print('encrypted result: ',encrypted_result2)       


the polynomial evaluated at r:  7.992729468039183e+30
encrypted result:  8
the result is: .... 1
